In [1]:
%matplotlib inline
import numpy as np
from scipy.integrate import solve_ivp
from parametres import Paras
from utilities import *
from initialisation import *
from odes import odes_scale_size
import matplotlib.pyplot as plt
from size_scaled_func import *
import datetime as dt
from simulation_func import *
import pandas as pd
import pickle as pkl
import os

# Running the Simulations

## Repeated runs of 30


- Setting 25 species, 50 resources
- 100 simulations were performed

In [2]:
# Setting parametres
N = 100
M = 100
#assemblenum = 1
para = Paras(N, M)
assemblies = 1

In [3]:
Rt_assemblies = []
Ct_assemblies = []
time_assemblies = []
para_assemblies = []

In [5]:
# Run simulation
start = dt.datetime.now()
for i in range(assemblies):
    Rt, Ct, t, para = sim_run(N, M, para, i, tstop=10e5, teval=10000)
    Rt_assemblies.append(Rt)
    Ct_assemblies.append(Ct)
    para_assemblies.append(para)
    time_assemblies.append(t)
    print(f'Asembly {i+1} complete, runtime:{dt.datetime.now()-start}') if (i+1) %5==0 else None

### Store the data

- Each Assembly will be stored in a dir called 'Abl_n' containing four files:
    - Resource concentration -- Abl_n_Rt.npy
    - Biomass concentration -- Abl_n_Ct.npy
    - Parametre -- Abl_n_Para.pkl
    - Time_series -- Abl_n_t.npy

In [6]:
if not os.path.exists('..\Data\Asb'):
    os.makedirs('..\Data\Asb')
for i in range(1, len(Ct_assemblies)+1):

    dir = f'..\Data\Asb\Abl_{i}'
    if not os.path.exists(dir):
        os.makedirs(dir) 

    with open(dir + f'\Abl_{i}_Ct.npy', 'wb') as f:
        np.save(f, Ct_assemblies[i-1])
    f.close()

    with open(dir + f'\Abl_{i}_Rt.npy', 'wb') as f:
        np.save(f, Rt_assemblies[i-1])
    f.close()

    with open(dir + f'\Abl_{i}_t.npy', 'wb') as f:
        np.save(f, time_assemblies[i-1])
    f.close()

    with open(dir + f'\Abl_{i}_Para.pkl', 'wb') as f:
        pkl.dump(para_assemblies[i-1], f)
    f.close()

## Subassemby runs

In [5]:
# Setting parametres
N = 100
M = 100
assemblenum = 1
assemblies = 30
subcommunity_size = 30
num_subcommunity = 30

In [6]:
R, C, t, para, idlist = sim_sub_run(N, M, assemblenum, 100000, 10000, subcommunity_size, num_subcommunity)

Initialisation completed!
Subcommunity simulation 5 completed runtime:0:05:08.267648
Subcommunity simulation 10 completed runtime:0:10:32.275424
Subcommunity simulation 15 completed runtime:0:15:54.104556
Subcommunity simulation 20 completed runtime:0:21:55.332622
Subcommunity simulation 25 completed runtime:0:27:27.060831
Subcommunity simulation 30 completed runtime:0:32:46.162688


### Load data structure
- data storage structure: FOLDER `sub_abl_beta=0.75` contains
    - FOLDER `subabl_{i}`, each of which contains the following:
        - Resource concentration -- `sAbl_n_Rt.npy`
        - Biomass concentration -- `sAbl_n_Ct.npy`
        - Parametre -- `sAbl_n_Para.pkl`
        - Time_series -- `sAbl_n_t.npy`

In [8]:
if not os.path.exists('..\Data\sub'):
    os.makedirs('..\Data\sub')
for i in range(1, num_subcommunity+1):

    dir = f'..\Data\sub\sAbl_{i}'
    if not os.path.exists(dir):
        os.makedirs(dir) 

    with open(dir + f'\sAbl_{i}_Ct.npy', 'wb') as f:
        np.save(f, C[i-1])
    f.close()

    with open(dir + f'\sAbl_{i}_Rt.npy', 'wb') as f:
        np.save(f, R[i-1])
    f.close()

    with open(dir + f'\sAbl_{i}_t.npy', 'wb') as f:
        np.save(f, t[i-1])
    f.close()

    with open(dir + f'\sAbl_{i}_Para.pkl', 'wb') as f:
        pkl.dump(para[i-1], f)
    f.close()

    with open(dir + f'\sAbl_{i}speciesID.npy', 'wb') as f:
        np.save(f, idlist[i-1])
    f.close()

## Subassemby runs with varying w

- let `w=1` to `w=2` from uniform to left skewed distribution
- Assuming each community has `subcommunity_size = 20` species selected from a pre-initialised species pool, each session consists of 30 communities for each value of `w`, where `w` will be binned into `50` values.
- Modify `sim_sub_run` function, then we store our results

In [2]:
## get w values
w = np.linspace(1, 2, 50)

## initialised data storage
Rw = []
Cw = []
tw = []
paraw = []
idlist = []

## initialised parametres
N = 100
M = 50
assemblenum = 1
assemblies = 30
subcommunity_size = 20
num_subcommunity = 30

In [10]:
### initialised parametres to ensure each session uses the same set of species
para = Paras(N, M)
p, number = int_preferences(N, M, para.mu_c, assemblenum)
D = int_conversion(M, para.Dbase, para.number, assemblenum, para.sparsity)
l = int_l(M, para.l_base, assemblenum)
rho = int_rho(M, para.rho_base, assemblenum)
vmax = int_vmax(N, M, para.v_max_base, p, number, assemblenum)
m = int_mt(N, para.m_base, assemblenum)

In [22]:
def sim_sub_run(N, M, assemblenum, tstop, teval, subcommunity_size=10, num_subcommunity=20, scale=True, w=2):

    # data storage
    R_assembles = []
    C_assembles = []
    t_assembles = []
    para_assembles = []
    identifiers = []
    model = Paras(N, M)
    avgm_global = allocate_avgm(N, w, assemblenum, model.ymin, model.ymax)

    # print(f'Initialisation completed!')
    # start = dt.datetime.now()

    # run sub_communities
    for i in range(num_subcommunity):

        para = Paras(subcommunity_size, M)
        ## select corresponding species for subcommunity assemblies
        rng = default_rng(seed=assemblenum+i+8*i)
        idx = rng.choice(range(N), size=subcommunity_size, replace=False)
        identifiers.append(idx)

        # reset and initialised
        new_p = p[idx, :]
        new_vmax = vmax[idx, :]
        new_m = m[idx, :]
        para.w = w
        avgm = avgm_global[idx]
        # print(avgm)
        ## Initialised Initial conditions
        R0 = int_R(M, para.R0, assemblenum)
        C0 = int_C(avgm, para.x0)

       # Load parametres
        para.paras(C0, R0, l, rho, new_p, new_vmax, new_m, D, avgm)
        time = np.linspace(0, tstop, teval)
        y0 = np.concatenate((R0, C0)).reshape(M+subcommunity_size,) # initial conditions
        # run and store
        if scale:
            pars = (para.l, para.m, para.rho, para.mu, para.km, para.p, para.D, para.v_in_max, para.type, para.B0, para.M0, para.E0, para.alpha, para.beta, para.gamma, para.R_half, para.avgm)
            result = solve_ivp(
            odes_scale_size, t_span=[time[0], time[-1]], y0=y0, t_eval=time, args=pars, dense_output=True)
        
        if not scale:
            pars = (para.l, para.m, para.rho, para.mu, para.km, para.p, para.D, para.v_in_max, para.type, para.R_half)
            result = solve_ivp(odes_not_scaled, t_span=[time[0], time[-1]], y0=y0, t_eval=time, args=pars, dense_output=True)
        
        Rt = result['y'][0:M]
        Ct = result['y'][M:M+subcommunity_size]
        t = result['t']

        # print(f'Subcommunity simulation {i+1} completed runtime:{dt.datetime.now()-start}') if (i+1)%5==0 else None

        R_assembles.append(Rt)
        C_assembles.append(Ct)
        t_assembles.append(t)
        para_assembles.append(para)

    return R_assembles, C_assembles, t_assembles, para_assembles, identifiers

In [23]:
# start running

start = dt.datetime.now()
for i, vals in enumerate(w):
    R, C, t, parat, idxs = sim_sub_run(N, M, i, 1000000, 10000, subcommunity_size, num_subcommunity, True, vals)
    Rw.append(R)
    Cw.append(C)
    tw.append(t)
    paraw.append(parat)
    idlist.append(idxs)
    
    print(f'Subcommunity simulation {i+1} completed runtime:{dt.datetime.now()-start}, w: {parat[0].w}')  if (i+1)%5==0 else None


Subcommunity simulation 5 completed runtime:0:09:42.743656, w: 1.0816326530612246
Subcommunity simulation 10 completed runtime:0:20:49.418348, w: 1.183673469387755
Subcommunity simulation 15 completed runtime:0:33:19.030816, w: 1.2857142857142856
Subcommunity simulation 20 completed runtime:0:46:44.858641, w: 1.3877551020408163
Subcommunity simulation 25 completed runtime:1:01:17.858204, w: 1.489795918367347
Subcommunity simulation 30 completed runtime:1:15:15.870946, w: 1.5918367346938775
Subcommunity simulation 35 completed runtime:1:29:33.073026, w: 1.693877551020408
Subcommunity simulation 40 completed runtime:1:45:58.217434, w: 1.7959183673469385
Subcommunity simulation 45 completed runtime:2:03:47.493588, w: 1.8979591836734693
Subcommunity simulation 50 completed runtime:2:18:56.374703, w: 2.0


- data storage structure: FOLDER `variedC` contains
    - FOLDER `w_val`, each of which contains the following:
        - Resource concentration -- `sAbl_n_Rt.npy`
        - Biomass concentration -- `sAbl_n_Ct.npy`
        - Parametre -- `sAbl_n_Para.pkl`
        - Time_series -- `sAbl_n_t.npy` $\newline$
for every n

In [33]:
## data storage
if not os.path.exists('..\Data\\variedC'):
    os.mkdir('..\Data\\variedC')

for i, val in enumerate(w):

    if 40 <= i < 50:

        dir = f'..\Data\\variedC\w_{i+1}'
        if not os.path.exists(dir): os.mkdir(dir)
        
        R = Rw[i]
        C = Cw[i]
        t = tw[i]
        para = paraw[i]
        idl = idlist[i]

        for j in range(1, len(R)+1):
            with open(dir + f'\w{i+1}_sAbl_{j}_Ct.npy', 'wb') as f:
                np.save(f, C[j-1])
            f.close()

            with open(dir + f'\w{i+1}_sAbl_{j}_Rt.npy', 'wb') as f:
                np.save(f, R[j-1])
            f.close()

            with open(dir + f'\w{i+1}_sAbl_{j}_t.npy', 'wb') as f:
                np.save(f, t[j-1])
            f.close()

            with open(dir + f'\w{i+1}_sAbl_{j}_Para.pkl', 'wb') as f:
                pkl.dump(para[j-1], f)
            f.close()

            with open(dir + f'\w{i+1}_sAbl_{j}_speciesID.npy', 'wb') as f:
                np.save(f, idl[j-1])
            f.close()

    else:
        pass
